# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Цель исследования:

1) Ответить на вопросы: 
- Существует зависимость между наличием детей и возвратом кредита в срок.
- Существует зависимость между семейным положением и возвратом кредита в срок.
- Существует зависимость между уровнем дохода и возвратом кредита в срок.
- Сущестует зависимость между целью получения кредита и возвратом кредита в срок.

2) Описать характер (как влияет фактор на срок возврата долга) для каждой подвержденной зависимости




## Шаг 1. Откройте файл с данными и изучите общую информацию

Импортируем необходимые библиотеки 

In [1]:
import pandas as pd 
from pymystem3 import Mystem
from collections import Counter
from IPython.display import display

In [2]:
scoring_data_native = pd.read_csv('/datasets/data.csv')# формируем DataFrame. 
scoring_data = scoring_data_native

if len(scoring_data) == 0: #проверка на путой датафрейм
    print('Dataframe пуст!')
m = Mystem() 

Далее будем работать с датафреймом в переменной scoring_data, для того, что бы всегда иметь доступ к исходным данным.

In [3]:
scoring_data.info()

#Подсчет количества строк с пропусками
print('\r\nКоличество пропусков в столбце days_employed :',scoring_data[scoring_data['days_employed'].isnull()]['purpose'].count(),'\r\n')# purpose взяли т.к. интересует любо столбец без пропусков 
print('Количество пропусков в столбце total_income :',scoring_data[scoring_data['total_income'].isnull()]['purpose'].count(),'\r\n')


print(scoring_data[scoring_data['days_employed'].isnull()]['income_type'].value_counts())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB

Количество пропусков в столбце days_employed : 2174 

Количество пропусков в столбце total_income : 2174 

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64


**Вывод**

В столбцах "days_employed" и "total_income" одинаковое количество пропущенных значений значений: 2174. Вероятно, эти пропуски связаны с тем, что при сборе данных сотрудники могли сделать ошибку и записать дробное число через запятую, а не точку и значение не записалось. 

Это подтверждает то, что люди трудоустроены, а значит должны иметь стаж и ежемесячный доход. В столбце "income_type" встречаются разные типы занятости, что говорит о том, что пропуски встречаются не только у каких-то конкретных типов.

Пропусков "None" нет в столбцах с пропусками. Об это говорит то, что тип данных в двух столбцах float64, а не object. 

## Шаг 2. Предобработка данных

### Обработка пропусков

Количество пропусков составляет около 10%, потерять такое количество записей мы не можем. Пропуски находятся в количественных переменных, поэтому отсортируем таблицу по этим столбцам и посмотрим на данные в первых и последних строках, что бы выбрать метод для заполнения : медианой или же средним арифметическим.

####  Обработка пропусков в столбце  "total_income"

Что бы заполнить пропуски наиболее приближенными значениями к действительности, будем опираться на тип занятости.
Посмотрим максимальные и минимальные значения для основных типов занятости. 

In [4]:
print(scoring_data['income_type'].value_counts())
print()    
print('Медиана ежемесячного дохода среди всех записей: {:.2f}'.format(scoring_data['total_income'].mean()))
print()
print('Максимальная ЗП типа занятости "сотрудник": {:.2f}'.format(scoring_data[scoring_data['income_type'] == 'сотрудник']['total_income'].max()))
print('Минимальная ЗП типа занятости "сотрудник": {:.2f}'.format(scoring_data[scoring_data['income_type'] == 'сотрудник']['total_income'].min()))
print()
print('Максимальная ЗП типа занятости "компаньон": {:.2f}'.format(scoring_data[scoring_data['income_type'] == 'компаньон']['total_income'].max()))
print('Минимальная ЗП типа занятости "компаньон": {:.2f}'.format(scoring_data[scoring_data['income_type'] == 'компаньон']['total_income'].min()))
print()
print('Максимальная ЗП типа занятости "госслужащий": {:.2f}'.format(scoring_data[scoring_data['income_type'] == 'госслужащий']['total_income'].max()))
print('Минимальная ЗП типа занятости "госслужащий": {:.2f}'.format(scoring_data[scoring_data['income_type'] == 'госслужащий']['total_income'].min()))
print()
print('Максимальная ЗП типа занятости "пенсионер": {:.2f}'.format(scoring_data[scoring_data['income_type'] == 'пенсионер']['total_income'].max()))
print('Минимальная ЗП типа занятости "пенсионер": {:.2f}'.format(scoring_data[scoring_data['income_type'] == 'пенсионер']['total_income'].min()))

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64

Медиана ежемесячного дохода среди всех записей: 167422.30

Максимальная ЗП типа занятости "сотрудник": 1726276.01
Минимальная ЗП типа занятости "сотрудник": 21367.65

Максимальная ЗП типа занятости "компаньон": 2265604.03
Минимальная ЗП типа занятости "компаньон": 28702.81

Максимальная ЗП типа занятости "госслужащий": 910451.47
Минимальная ЗП типа занятости "госслужащий": 29200.08

Максимальная ЗП типа занятости "пенсионер": 735103.27
Минимальная ЗП типа занятости "пенсионер": 20667.26


Безработных, декретных и студентов рассматривать не будем, т.к. их количество незначительно и для них подойдет любой выбранный способ.

Из данных видно, что разрыв между минимальным и максимальным значением для каждого типа занятости очень большой и среднее значение плохо характеризует по-настоящему средний доход, поэтому пропуски будем заполнять медианой среди значений соответствующих типов занятости.

In [5]:
#заполняем пропуски соотв. медианами
scoring_data['total_income'] = scoring_data.groupby('income_type')['total_income'].apply(lambda x: x.fillna(x.median()))
scoring_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Пропуски в столбце total_income заполнены. 

#### Обработка пропусков в столбце "days_employed"

Для поставленных целей данный столбец, вероятно, не понадобится, поэтому не будем применять для него более подробный анализ для заполнения пропусков. Применим метод abs() для возведения в модуль значений столбца и избавления от отрицательных значений, а затем заполним пропуски медианой среди всех значений.  

In [6]:
#сначала избавимся от отрицательных значений, чтобы верно посчитать медиану
scoring_data['days_employed'] = scoring_data['days_employed'].abs()
#заполним пропуски
scoring_data['days_employed'] = scoring_data['days_employed'].fillna(scoring_data['days_employed'].median())
scoring_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


После заполнения всех пропусков, при оценке значений категориальных переменных среди значимых (отностиельно целей исследования) показателей обнаружились отрицательные значения в столбце "children". 

In [7]:
print(scoring_data['children'].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


Что бы избавится от отрицательных значений в столбце применим к нему метод abs().

Также в столбце был обнаружен вероятный артефакт: 76 выданных кредитов людям, у которых по **20 детей**. Рекомендуется проверить, как попадают эти данные в датасет, предположительно ошибка может возникнуть из-за опечатки сотрудников (лишний ноль в конце).
Т.к. мы проверяем гипотезу о наличии зависимости между наличием детей и возвратом в срок, нам может быть интересно посмотреть зависимость и от количества детей. Однако количество таких строк 0,35% и вряд ли они повлияют на результат исследования, поэтому оставим данные как есть.

In [8]:
scoring_data['children'] = scoring_data['children'].abs() 

**Вывод**
В таблице больше нет пропусков, артефактов, которые нам помешают на первый взгляд нет и можно двигаться дальше.

Артефакты в значениях столбца "days_eployed"(стаж 1000 лет или отрицательные значения) могут быть связаны с неправильно составленной формулой способа исчисления трудового стажа, установленного ФЗ No 255-ФЗ. Необходимо проверить, каким образом эти данные попадают в таблицу 

Также в таблице 101 запись с параметром "dob_years" равным нулю. Для поставленных целей возраст заемщика, вероятно, не понадобится, поэтому исправлять данный артефакт оставим без внимания. Однако по этому артефакту также стоит проверить сбор данных, т.к. если в дальнейшем он понадобится для анализа, то заполнять нулевые значения средними значениями возвраста не будет смысла, а зависмость возвраста от стажа или типа занятости крайне сомнительна.

Интересно, что безработным, декретным и студентам очень редко все же одобряются кредиты, в основном на цели, связанные с жильем, хотя риск очень большой. Возможно роль сыграл неплохой ежемесячный доход.

### Замена типа данных

Т.к. в данном случае разница в рубль в ежемесячном доходе для нас не принципиальна, поэтоу оруглять значения не будем и просто отбросим дробную часть. 
Трудовой стаж, как правило, округляется в большую сторону, поэтому применим округление до ближайшего четного для значений столбца "days_employed".

In [9]:
scoring_data['days_employed'] = scoring_data['days_employed'].apply(lambda x: round(x))
scoring_data['total_income'] = scoring_data['total_income'].astype(int)
print(scoring_data['total_income'].head())
print(scoring_data['days_employed'].head())

0    253875
1    112080
2    145885
3    267628
4    158616
Name: total_income, dtype: int64
0      8438
1      4025
2      5623
3      4125
4    340266
Name: days_employed, dtype: int64


In [10]:
scoring_data['total_income'].round().astype(int)

0        253875
1        112080
2        145885
3        267628
4        158616
          ...  
21520    224791
21521    155999
21522     89672
21523    244093
21524     82047
Name: total_income, Length: 21525, dtype: int64

**Вывод**

В результате оба столбца имеют тип данных int64, значит, преобразование прошло. В случае необходимости округления по математическм правилам можно было бы написать свою функцию.

### Обработка дубликатов

С помощью value_counts() найдем количество уникальных значений в каждом столбце. Не используем unique(), т.к., возможно, потребуется оценить также количество таких значений.  

In [11]:
def find_uniq_values(series):
    return print(series.value_counts(),'\r\n')
scoring_data.apply(find_uniq_values)

0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64 

2194      2179
200         15
1482        14
206         14
220         14
          ... 
7739         1
368193       1
400962       1
7748         1
347045       1
Name: days_employed, Length: 9058, dtype: int64 

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64 

среднее                13750
высшее                  4718


children            None
days_employed       None
dob_years           None
education           None
education_id        None
family_status       None
family_status_id    None
gender              None
income_type         None
debt                None
total_income        None
purpose             None
dtype: object

В столбце "education" найдены значения, которые отличаются между собой регистром. Перед проверкой на дубли и их очисткой приведем значеня в единый регистр. Значения в остальных столбцах в норме.

In [12]:
print(scoring_data['education'].value_counts())

scoring_data['education'] = scoring_data['education'].str.lower()
print()
print('После перевода в нижний регистр:\r\n')
print(scoring_data['education'].value_counts())

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

После перевода в нижний регистр:

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64


Проверим на дубликаты датасет. Использовать будем метод duplicated() для того, что бы найти дубли строк во всем датасете. 

In [13]:
print('Количество одинаковых строк:',scoring_data.duplicated().sum())
   

Количество одинаковых строк: 71


В данном случае дубли могли появится из-за заполнения пропущенных данных  медианами. Однако количество дублей всего 71 шт (0,33%), поэтому удалим их методом drop_duplicates() 

In [14]:
scoring_data = scoring_data.drop_duplicates().reset_index(drop = True)
print('Количество одинаковых строк:',scoring_data.duplicated().sum())
print()
scoring_data.info()

Количество одинаковых строк: 0

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
children            21454 non-null int64
days_employed       21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null int64
purpose             21454 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Дубли очищены, количество строк в таблице уменьшилось ровно на количество дублей.
Заказчику рекомендуется обратиться в отдел разработки с предложением обработки таких значений до записи в датасет: приведение к единому регистру либо дать пользователям возможность выбрать из предопределенных вариантов.

### Лемматизация

Выделим леммы из каждого значения в столбце "purpose" в столбец "lemmas". Проанализируем, сколько всего разных целей среди выданных кредитов.

In [15]:
def find_lemmas(string):# функция для выделения лемм в строке
    lemmas = m.lemmatize(string)
    return lemmas

scoring_data['purpose_lemmas'] = scoring_data['purpose'].apply(find_lemmas)# в столбце lemmas будут хранится выделенные леммы
print('Найденные леммы:\r\n',scoring_data['purpose_lemmas'].value_counts())#найдем уникальные выделенные леммы 


Найденные леммы:
 [автомобиль, \n]                                          972
[свадьба, \n]                                             791
[на,  , проведение,  , свадьба, \n]                       768
[сыграть,  , свадьба, \n]                                 765
[операция,  , с,  , недвижимость, \n]                     675
[покупка,  , коммерческий,  , недвижимость, \n]           661
[операция,  , с,  , жилье, \n]                            652
[покупка,  , жилье,  , для,  , сдача, \n]                 651
[операция,  , с,  , коммерческий,  , недвижимость, \n]    650
[покупка,  , жилье, \n]                                   646
[жилье, \n]                                               646
[покупка,  , жилье,  , для,  , семья, \n]                 638
[строительство,  , собственный,  , недвижимость, \n]      635
[недвижимость, \n]                                        633
[операция,  , со,  , свой,  , недвижимость, \n]           627
[строительство,  , жилой,  , недвижимость, \n]      

Как видно из данных, уникальных лемм не так много и можно выделить пять главных групп целей кредитования:
1) покупка автомобиля
2) покупка недвижимости
3) получение образования
4) свадьба
5) ремонт жилья

**Вывод**

С помощью лемматизации выделены основные группы целей кредитования, по которым в дальнейшем будет производится категоризация.
Чаще всего кредит берутся для приобретения недвижимости, самая редкая цель кредитования - ремонт. 

В остальных признаках лемматизация не требуется, т.к. данные, интересующие нас, уже характеризуют категории заемщиков в необходимых аналитиках


### Категоризация данных

Для дальнейшей категоризации добавим к dataframe столбцы:

1) "have_child" - признак, принимающий значение "1", если у заемщика ребенок есть и "0" если ребенка нет

2) "total_income_category" - признак, характеризующий категории дохода. Для выделения категорий по данному принаку посмотрим медианы ежемесячного дохода для каждого типа занятости.

3) "purpose_category" - признак, характеризующий категории целей кредитования и принимающий соответсвующие значения: автомобиль, недвижимость, образование, ремонт, свадьба

In [16]:
print('Медиана дохода для типа занятости "сотрудник": {:.2f}'.format(scoring_data[scoring_data['income_type'] == 'сотрудник']['total_income'].median()))
print('Медиана дохода для типа занятости "пенсионер": {:.2f}'.format(scoring_data[scoring_data['income_type'] == 'пенсионер']['total_income'].median()))
print('Медиана дохода для типа занятости "компаньон": {:.2f}'.format(scoring_data[scoring_data['income_type'] == 'компаньон']['total_income'].median()))
print('Медиана дохода для типа занятости "госслужащий": {:.2f}'.format(scoring_data[scoring_data['income_type'] == 'госслужащий']['total_income'].median()))

Медиана дохода для типа занятости "сотрудник": 142594.00
Медиана дохода для типа занятости "пенсионер": 118514.00
Медиана дохода для типа занятости "компаньон": 172357.00
Медиана дохода для типа занятости "госслужащий": 150447.00


Для столбца "total_income_category" выделим следующие категории: 
- значение "низкий" при ежемесячном доходе менее 70 000 у.е 
- "ниже среднего" при ежемесячном доходе 70 000 - 120 000 у.е 
- "средний" при ежемесячном доходе 120 000 - 160 000 у.е. 
- "высокий" при ежемесячном доходе от 160 000 у.е. 

In [17]:
def have_child_category(child_value):#категоризация по налиию детей
    if child_value == 0:
        return 'детей нет'
    else:
        return 'дети есть'

def  set_income_category(income_value):#категоризация по ежемесячному доходу
    if income_value < 70000:
        return 'низкий'
    elif ((income_value >= 70000)&(income_value<120000)):
        return 'ниже среднего'
    elif ((income_value >= 120000)&(income_value<160000)):
        return 'средний'
    return 'высокий'

def set_purpose_category(purpose_values):#категоризация по целям кредитования
    if 'свадьба' in purpose_values:
        return 'свадьба'
    elif 'ремонт' in purpose_values:
        return 'ремонт'
    elif 'автомобиль' in purpose_values:
        return 'автомобиль'
    elif 'образование' in purpose_values:
        return 'образование'
    elif ((('жилье' in purpose_values) & ('ремонт' not in purpose_values)) or ('недвижимость' in purpose_values)):
        return 'недвижимость'
    else:    
        return 'неизвестно' #для проверки лемм, которые не попали ни в одну категорию.
    
scoring_data['have_child'] = scoring_data['children'].apply(have_child_category)
scoring_data['total_income_category'] = scoring_data['total_income'].apply(set_income_category)
scoring_data['purpose_category'] = scoring_data['purpose_lemmas'].apply(set_purpose_category)

if (scoring_data[scoring_data['purpose_category'] == 'неизвестно'].empty):#проверяем, все ли леммы попали в категории
    print('Все леммы попали в категории!')
print()
display(scoring_data.head(10))

Все леммы попали в категории!



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas,have_child,total_income_category,purpose_category
0,1,8438,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",дети есть,высокий,недвижимость
1,1,4025,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",дети есть,ниже среднего,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",детей нет,средний,недвижимость
3,3,4125,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",дети есть,высокий,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",детей нет,средний,свадьба
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,"[покупка, , жилье, \n]",детей нет,высокий,недвижимость
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,"[операция, , с, , жилье, \n]",детей нет,высокий,недвижимость
7,0,153,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,"[образование, \n]",детей нет,средний,образование
8,2,6930,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,"[на, , проведение, , свадьба, \n]",дети есть,ниже среднего,свадьба
9,0,2189,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]",детей нет,средний,недвижимость


**Вывод**

Все необходимые признаки для анализа категоризированы: семейное положение, ежемесячный доход, наличие детей и цель кредита.

Цифры об общем ежемесячном доходе кажутся сильно завышенными как минимум для пенсионеров и госслужащих. Значения сильно больше значений как официальной статистики, так и так  посчитанной сторонними организациями.  Возможно, стоит проверить достоверность данных и пересмотреть границы соответсвующих категорий.
Такой предположительный артефакт может быть связан с тем, что банк допускает указывать незадекларированный доход в общем ежемесячном доходе, а заемщики этим пользуются, для увеличения вероятности одобрения займа. В таком случае можно предложить заказчику разделить общий ежемесячный доход на декларируемый и недекларируемый, для более объективных данных.

## Шаг 3. Ответьте на вопросы

### Есть ли зависимость между наличием детей и возвратом кредита в срок?

Выдвинем гипотезу.

**Гипотеза 1. 
Заемщики, у которых есть дети, реже становятся должниками.** 

Предполагаемые причины: т.к. наличие детей дисциплинирует, а также заемщики больше опасаются негативных последствий при просроченной задолженности, которые могут повлиять также и на детей (арест счетов, опись имущества, коллекторские мероприятия и т.д.)

Для проверки гипотезы проанализируем:

1) процент невозврата кредитов у заемщиков с детьми и без детей

2) количество выданных кредитов заемщикам с детьми и без детей. Возможно, что объем выборки может быть слишком маленькой  для подвтерждения или опровержения гипотезы

Также будет интересно посмотреть процент невозврата для заемщиков с разным колиеством детей

In [18]:
print('Количество кредитов, выданных заемщикам с детьми:',scoring_data[scoring_data['have_child'] == 'дети есть']['have_child'].count())
print('Количество кредитов, выданных заемщикам без детей:',scoring_data[scoring_data['have_child'] == 'детей нет']['have_child'].count())

total_debt_value = (scoring_data[scoring_data['debt'] == 1]['debt'].count())/len(scoring_data)#общий процент невозврата
debt_with_child = (scoring_data[(scoring_data['debt'] == 1) & (scoring_data['have_child'] == 'дети есть')]['debt'].count())/(scoring_data[scoring_data['have_child'] == 'дети есть']['debt'].count())#общий процент невозврата
debt_without_child = (scoring_data[(scoring_data['debt'] == 1) & (scoring_data['have_child'] == 'детей нет')]['debt'].count())/(scoring_data[scoring_data['have_child'] == 'детей нет']['debt'].count())#общий процент невозврата
print()
print('Общий процент невозврата: {:.1%}'.format(total_debt_value))
print('Процент невозврата заемщиками, у которых есть дети: {:.1%}'.format(debt_with_child))
print('Процент невозврата заемщиками, у которых нет детей: {:.1%}'.format(debt_without_child))

debt_number_of_childrens = scoring_data.groupby('children').sum()#группируем датасет по количеству детей
print()

#находим количество займов для заемщиков с определенным количествомм детей
debt_number_of_childrens['credit_count'] = scoring_data.groupby('children')['debt'].count()

#находим медиану среднего дохода для заемщиков с определенным колиеством детей
debt_number_of_childrens['income_median'] = scoring_data.groupby('children')['total_income'].median()

#находим процент невозврата займов для заемщиков с определенным количествомм детей
debt_number_of_childrens['debt_percent'] = round((scoring_data.groupby('children')['debt'].sum()/debt_number_of_childrens['credit_count'])*100,1)
display(debt_number_of_childrens[['debt','credit_count','debt_percent','income_median']])
print()
display(scoring_data.groupby('children')['purpose_category'].value_counts())

Количество кредитов, выданных заемщикам с детьми: 7363
Количество кредитов, выданных заемщикам без детей: 14091

Общий процент невозврата: 8.1%
Процент невозврата заемщиками, у которых есть дети: 9.2%
Процент невозврата заемщиками, у которых нет детей: 7.5%



,debt,credit_count,debt_percent,income_median
children,,,,
0,1063,14091,7.5,142594.0
1,445,4855,9.2,145521.0
2,194,2052,9.5,142594.0
3,27,330,8.2,145220.0
4,4,41,9.8,150447.0
5,0,9,0.0,168460.0
20,8,76,10.5,144036.5


children  purpose_category
0         недвижимость        6672
          автомобиль          2845
          образование         2642
          свадьба             1530
          ремонт               402
1         недвижимость        2334
          автомобиль           974
          образование          876
          свадьба              534
          ремонт               137
2         недвижимость         978
          образование          403
          автомобиль           399
          свадьба              215
          ремонт                57
3         недвижимость         160
          образование           69
          автомобиль            60
          свадьба               32
          ремонт                 9
4         недвижимость          21
          автомобиль            10
          образование            7
          свадьба                3
5         недвижимость           5
          автомобиль             2
          образование            1
          свадьба           

**Вывод**

Выдвинутая гипотеза отвергнута. Процент невозврата у заемщиков с детьми оказался выше, чем у заемщиков без детей (9,2% против 7,5%) и даже оказался выше, чем общий процент невозврата. **Это говорит о том, что связь между наличием детей и возвратом кредита в срок есть: заемщики с детьми чаще допускают просрочки по кредиту, чем заемщики без детей.**

Также при рассмотрени целей кредита для заемщиков с разным количеством детей видно, что наличия или отсутсвия детей тройка самых популярных целей одна и та же: недвижимость, авто, образование. Это говорит о том, что влияние количества детей на цель кредита либо исключено либо минимально и не может быть косвенной причиной полученных результатов.

Одна из вероятных причин может быть связана с тем, что при наличии детей, заемщик несет повышенные расходы и из-за этого может допускать просрочки по кредиту. Это подтверждает то, что с ростом количества детей процент невозврата растет, а ежемесячный доход остается на прежнем уровне.



### Есть ли зависимость между семейным положением и возвратом кредита в срок?

Выдвинем гипотезы:

**Гипотеза 2.  Самый низкий процент невозврата у заемщиков, которые либо находятся в зарегистрированных отношениях или в гражданском браке**
Возможные причины: т.к. в таком случае долговая нагрузка может распределятся на двух человек. 

**Гипотеза 3. Самый высокий процент невозврата у заемщиков, которые являются вдовами/вдовцами или же находящимися в разводе.**
Возможные причины: эмоциональное состояние и отсутсвие возможности распределения долговой нагрузки.

При наличии категории "гражданский брак" полагаем, что, если заемщик рзведен, но живет с кем-то в гражданском браке, то он все равно попадает в категорию с семейным положением "гражданский брак".

In [19]:
print('Количество кредитов для категорий:')
print(scoring_data['family_status'].value_counts())
print()

for_family_status = scoring_data.groupby('family_status').sum()#группируем датасет по семейному положению
print()

#находим количество займов для заемщиков с определенным семейным положением
for_family_status['credit_count'] = scoring_data.groupby('family_status')['debt'].count()

#находим медиану среднего дохода для заемщиков с определенным семейным положением
for_family_status['income_median'] = scoring_data.groupby('family_status')['total_income'].median()

#находим процент невозврата займов для заемщиков с определенным семейным положением
for_family_status['debt_percent'] = round((scoring_data.groupby('family_status')['debt'].sum()/for_family_status['credit_count'])*100,1)
display(for_family_status[['debt','credit_count','debt_percent','income_median']])
print()
display(scoring_data.groupby('family_status')['purpose_category'].value_counts())
print()
print('Среднее значение ежемесячного дохода для каждой категории признака "семейное положение"',for_family_status['income_median'].mean())

Количество кредитов для категорий:
женат / замужем          12339
гражданский брак          4151
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64




,debt,credit_count,debt_percent,income_median
family_status,,,,
Не женат / не замужем,274,2810,9.8,142594
в разводе,85,1195,7.1,143450
вдовец / вдова,63,959,6.6,127310
гражданский брак,388,4151,9.3,142594
женат / замужем,931,12339,7.5,142969


family_status          purpose_category
Не женат / не замужем  недвижимость        1503
                       автомобиль           637
                       образование          577
                       ремонт                93
в разводе              недвижимость         637
                       автомобиль           281
                       образование          238
                       ремонт                39
вдовец / вдова         недвижимость         510
                       автомобиль           218
                       образование          199
                       ремонт                32
гражданский брак       свадьба             2324
                       недвижимость         934
                       автомобиль           434
                       образование          404
                       ремонт                55
женат / замужем        недвижимость        6620
                       автомобиль          2736
                       образование         2595



Среднее значение ежемесячного дохода для каждой категории признака "семейное положение" 139783.4


**Вывод**

**Связь между семейным положением и возвратом кредита в срок есть:**

Несмотря на то, что у заемщиков с семейным положением "вдова/вдовец" средний доход на ~9% меньше, чем средний доход заемщиков в разных семейных положениях, процент невозврата для кредитов, выданных им оказался самым низким - 6,6%. **Это означает, что заемщики с семейным положением "вдова/вдовец" реже остальных допускают просрочки по кредиту.**

Процент невозврата кредитов, выданных заемщикам с семейным положением "в разводе" также оказался одним из самых низких - 7.1%
**Это означает, что заемщики с семейным положением "в разводе" допускают просрочки по кредиту реже тех, кто находится в зарегистрированных отношениях, гражданском браке и заемщиков, с семейным положением "не женат/не замужем".**
Следовательно, гипотеза №3 отвергнута.
Из имеющихся данных сложно предположить причины таких результатов, возможно ответ кроется в психологии?

**У заемщиков с семейным положением "женат/замужем" оказался ниже, чем у не женатых заемщиков или находящихся в гражданском браке - 7,5%. Однако процент невозврата у живущих в гражданском браке заемщиков один из самых высоких - 9.3%.**
Следовательно, гипотеза №2 частично подтверждена.

**Самый высокий процент невозврата кредитов у заемщиков с семейным положением "не женат/не замужем" - 9,8%**

Интересно, что для заемщиков популярные цели кредита почти одинаковы, кроме заемщиков в семейном положении "в гражданском браке" - они, в основном, берут кредиты для того, что бы провести свадьбу.  


### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Выдвинем гипотезу:

**Гипотеза 4. Чем выше доход, тем меньше процент невозврата кредитов.**


In [20]:
print('Количество кредитов для категорий:')
print(scoring_data['total_income_category'].value_counts())
print()

debt_income_category = scoring_data.groupby('total_income_category').sum()#группируем датасет по уровню дохода
print()

#находим количество займов для заемщиков с определенным уровнем дохода
debt_income_category['credit_count'] = scoring_data.groupby('total_income_category')['debt'].count()

#находим медиану среднего дохода для заемщиков с определенным уровнем дохода
debt_income_category['income_median'] = scoring_data.groupby('total_income_category')['total_income'].median()

#находим процент невозврата займов для заемщиков с определенным уровнем дохода
debt_income_category['debt_percent'] = round((scoring_data.groupby('total_income_category')['debt'].sum()/debt_income_category['credit_count'])*100,1)
display(debt_income_category[['debt','credit_count','debt_percent','income_median']])
print()
display(scoring_data.groupby('total_income_category')['purpose_category'].value_counts())
print()

Количество кредитов для категорий:
высокий          8720
ниже среднего    5757
средний          5503
низкий           1474
Name: total_income_category, dtype: int64




,debt,credit_count,debt_percent,income_median
total_income_category,,,,
высокий,668,8720,7.7,212990.5
ниже среднего,485,5757,8.4,99022.0
низкий,101,1474,6.9,58542.5
средний,487,5503,8.8,142594.0


total_income_category  purpose_category
высокий                недвижимость        4207
                       автомобиль          1739
                       образование         1582
                       свадьба              927
                       ремонт               265
ниже среднего          недвижимость        2726
                       автомобиль          1142
                       образование         1078
                       свадьба              668
                       ремонт               143
низкий                 недвижимость         695
                       автомобиль           299
                       образование          289
                       свадьба              150
                       ремонт                41
средний                недвижимость        2576
                       автомобиль          1126
                       образование         1064
                       свадьба              579
                       ремонт               158


**Вывод**

Вопреки ожиданиям, самый низкий процент невозврата оказался у заемщиков с низким уровнем дохода - 6,9%, далее идут заемщики с высоким уровнем дохода, у которых процент невозврата 7.7%. Самый высокий процент невозврата у заемщиков с доходом ниже среднего и средним - 8,4% и 8,8% соответсвенно.

**Это говорит о том, что связь между уровнем дохода есть, однако она нелинейная - с увеличением уровня дохода процент невозврата не падает.** Самые надеждные заемщики это заемщики с низким и высоким уровнями доходов.
Следовательно, гипотеза №4 отвергнута.

### Как разные цели кредита влияют на его возврат в срок?

Выдвинем гипотезу:

**Гипотеза 5. Реже всего заемщики не вовращают кредиты выданные на недвижимость.**
Возможные причины: в случае невозврата кредита у заемщика могут жилье, что стимулирует его к возврату кредита.

In [21]:
print('Количество кредитов для категорий:')
print(scoring_data['purpose_category'].value_counts())
print()

debt_purpose_category = scoring_data.groupby('purpose_category').sum()#группируем датасет по уровню дохода
print()

#находим количество займов для заемщиков с определенным уровнем дохода
debt_purpose_category['credit_count'] = scoring_data.groupby('purpose_category')['debt'].count()

#находим медиану среднего дохода для заемщиков с определенным уровнем дохода
debt_purpose_category['income_median'] = scoring_data.groupby('purpose_category')['total_income'].median()

#находим процент невозврата займов для заемщиков с определенным уровнем дохода
debt_purpose_category['debt_percent'] = round((scoring_data.groupby('purpose_category')['debt'].sum()/debt_purpose_category['credit_count'])*100,1)
display(debt_purpose_category[['debt','credit_count','debt_percent','income_median']])


Количество кредитов для категорий:
недвижимость    10204
автомобиль       4306
образование      4013
свадьба          2324
ремонт            607
Name: purpose_category, dtype: int64




,debt,credit_count,debt_percent,income_median
purpose_category,,,,
автомобиль,403,4306,9.4,142594.0
недвижимость,747,10204,7.3,142617.5
образование,370,4013,9.2,142594.0
ремонт,35,607,5.8,148925.0
свадьба,186,2324,8.0,142594.0


**Вывод**

**Связь между целью кредита и возвратом кредита, выданного для этой цели есть: реже всего не возвращают кредиты ремонт и , как ожидалось, недвижимость - процент невозврата 5.8% и 7.3% соответсвенно.** Следовательно, гипотеза №5 подтверждена частично. Кредитов на ремонт выдано не так много, возможно это также одна из причин низкого уровня невозврата.

## Шаг 4. Общий вывод

В результате исследования были сделаны следующие выводы:

1) Заемщики с детьми чаще допускают просрочки по кредиту, чем заемщики без детей.

2) Самые надежные заемщики находятся в семейном положении "в разводе" и "женат/замужем". Самые ненадежные заемщики с семейным положением "не женат/не замужем".

3) Самые надеждные заемщики это заемщики с низким и высоким уровнями доходов.

4) Реже всего не возвращают кредиты, взятые на ремонт и недвижимость. Чаще всего не возвращают кредиты  на автомобиль и образование.

Из пяти гипотез, которые были выдвинуты без изучения статистики, три были отвергнуты и две подтверждены только частично, хотя и казались логичными, поэтому при принятии важных решений лучше опираться на результаты анализа, основанного на статистике и фактах.

Заказчику рекомендуется обратиться в отдел разработки, для того что бы выяснить, почему в датасете появляются артефакты и пустые значения. Если возможно устранить пропуски и артефакты в интересующей выборке, то рекомендуется проанализировать новый исправленный датасет, что бы убедится в корректности результатов исследования.

При наличии дополнительных данных можно сделать более глубокий анализ. Можно рассмотреть как влияют на возврат кредита в срок такие показатели,как, например: сумма кредита; срок просрочки